# Cats vs Dogs - Binary Image Classifier

Import Libraries

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm

Load Dataset

In [124]:
class CatsvsDogs(Dataset):
    def __init__(self, data_path, df, transform=None):
        self.data_path = data_path
        self.transform = transform
        
        self.img_path = df.iloc[:,0].tolist()
        self.img_label = df.iloc[:,1].tolist()
    
    def __len__(self):
        return len(self.img_label)

    def __getitem__(self, idx):
        file_name = self.img_path[idx]
        label = self.img_label[idx]

        img = Image.open(os.path.join(self.data_path, file_name))

        if self.transform:
            img = self.transform(img)

        return img, label

In [377]:
interpolation_mode = transforms.InterpolationMode.BICUBIC
resize_transform = transforms.Resize((150,150), interpolation=interpolation_mode)

transform = transforms.Compose([
                            resize_transform,
                            transforms.Grayscale(num_output_channels=1),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,)),
                        ])
    
data_path='PetImages'
file_path='catsvsdogs.csv'

img_data = pd.read_csv(file_path)

full_dataset = CatsvsDogs(data_path=data_path, df=img_data, transform=transform)

print("length of dataset:", full_dataset.__len__())


length of dataset: 24998


In [424]:
train_ratio = 0.8
test_ratio = 1.0 - train_ratio

all_data = len(full_dataset)
train_data = int(train_ratio * all_data)
test_data = all_data - train_data

train_dataset, test_dataset = random_split(full_dataset, [train_data, test_data])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("training images:", len(train_dataset))
print("testing images:", len(test_dataset))

training images: 19998
testing images: 5000


CNN

In [419]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, 1, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
    
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, 1, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
        
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, 1, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv(x)
        x = nn.functional.adaptive_avg_pool2d(x,(1,1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [420]:
net = Net()
print(net)

Net(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): ReLU()
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): ReLU(

Train

In [421]:
optimizer = optim.Adam(net.parameters(), lr=0.002)
loss_function = nn.MSELoss()

In [422]:
epochs = 20
net.train()
correct = 0
total = 0

for epoch in range(epochs):
    running_loss = 0.0
    for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        optimizer.zero_grad()
        
        output = net(x).reshape(-1)
        loss = loss_function(output, y.to(torch.float32))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        predicted = (output > 0.5).float() 

        correct += (predicted == y).sum().item()
        total += y.size(0)

    accuracy = correct / total if total > 0 else 0.0
    print(f'accuracy: {accuracy:.2%}')
    print("epoch:", (epoch + 1), "loss:", (running_loss))

print('done training.')

100%|█████████████████████████████████████████| 625/625 [43:55<00:00,  4.22s/it]


accuracy: 65.31%
epoch: 1 loss: 135.38058509677649


100%|█████████████████████████████████████████| 625/625 [42:34<00:00,  4.09s/it]


accuracy: 69.44%
epoch: 2 loss: 111.52807682007551


100%|█████████████████████████████████████████| 625/625 [42:38<00:00,  4.09s/it]


...

accuracy: 92.21%
epoch: 18 loss: 6.4762938438485165


100%|█████████████████████████████████████████| 625/625 [44:47<00:00,  4.30s/it]


accuracy: 92.54%
epoch: 19 loss: 8.455690973718625


100%|█████████████████████████████████████████| 625/625 [44:15<00:00,  4.25s/it]

accuracy: 92.84%
epoch: 20 loss: 7.162102483797739
done training.


Evaluate

In [423]:
net.eval()
correct = 0
total = 0

with torch.no_grad():
    for i, (x, y) in tqdm(enumerate(test_loader), total=len(test_loader)):
        output = net(x).reshape(-1)
        predicted = (output > 0.5).float() 

        correct += (predicted == y).sum().item()
        total += y.size(0)

accuracy = correct / total if total > 0 else 0.0
print(f'accuracy: {accuracy:.2%}')

100%|█████████████████████████████████████████| 157/157 [04:10<00:00,  1.59s/it]

accuracy: 90.60%
